# Objective 

Author: Kate Lassiter

Developed a Python visualization application to show predictions versus baseline 

In [ ]:
from shiny import App, reactive, render, ui
import matplotlib.pyplot as plt
import numpy as np
import asyncio

def simulate_acceleration_data(start_point, time_points):
    accel_data=[]
    activities=[]
    
    for i in range(start_point, start_point + time_points):
        if (i//200) % 2 == 0:
            accel = np.sin(i * 0.1) + np.random.normal(0, 0.1)
            activity = 'walking'
        else:
            accel = np.random.normal(0, 0.05)
            activity = 'not walking'
        accel_data.append(accel)
        activities.append(activity)
    return np.array(accel_data), activities

app_ui = ui.page_fluid(
    # ui.sidebar(
    #     ui.input_select("patient_id", "Select Patient ID", {str(i): str(i) for i in range(1, 29)}),
    #     ui.input_slider("time", "Time", 1, 10000, 500, step=500),  # Adjust slider to max 5000, step by 500
    #     ui.input_action_button("auto_scroll", "Start Auto Scroll")
    # ),
    ui.input_select("patient_id", "Select Patient ID", {str(i): str(i) for i in range(1, 29)}),
    ui.input_slider("time", "Time", 1, 50000, 500, step=500),  
    ui.input_action_button("auto_scroll", "Start Auto Scroll"),
    ui.output_plot("actual_plot"),   
    ui.output_plot("predicted_plot") 
)

def server(input, output, session):
    time_=reactive.Value(1)
    async def start_auto_scroll():
        for start_time in range(1, 50000, 500):  
            accel_data, activities = simulate_acceleration_data(start_time, 500)
            session.send_input_message("time", {"value": start_time + 500})
            time_.set(start_time + 500) 
            # for x in range(0,5):
            #     print(input.time(),"desired",start_time + 500)
            #     if input.time() != start_time + 500:
            #         print("fail")
            #         session.send_input_message("time", {"value": start_time + 500})
            #         time_.set(start_time + 500) 
            #         print(time_.get())
            #     else:
            #         break

            await asyncio.sleep(0.007) 
    
    @reactive.Effect
    @reactive.event(input.auto_scroll)
    def auto_scroll():
        asyncio.create_task(start_auto_scroll())

    @output
    @render.plot
    def actual_plot():
        #time_value = time_
        time_value = time_.get()
        start_time = time_value - 500 
        accel_data, activities = simulate_acceleration_data(start_time, 500)
        fig, ax = plt.subplots()
        x = np.arange(start_time, time_value)
        y = accel_data[:500]
        ax.plot(x, y, label="Acceleration")
        for i in range(500):
            color = 'green' if activities[i] == 'walking' else 'red'
            ax.axvspan(x[i]-0.5, x[i]+0.5, facecolor=color, alpha=0.3)
        
        ax.set_title(f"Actual Activity: Patient {input.patient_id()}")
        ax.legend()
        return fig

    @output
    @render.plot
    def predicted_plot():
        time_value = time_.get()
        start_time = time_value - 500 
        accel_data, activities = simulate_acceleration_data(start_time, 500)
        fig, ax = plt.subplots()
        x = np.arange(start_time, time_value)
        y = accel_data[:500] + np.random.normal(0, 0.02, size=500) 
        ax.plot(x, y, label="Predicted Acceleration")
        for i in range(500):
            color = 'red' if activities[i] == 'walking' else 'green'
            ax.axvspan(x[i]-0.5, x[i]+0.5, facecolor=color, alpha=0.3)
        ax.set_title(f"Predicted Activity: Patient {input.patient_id()}")
        ax.legend()
        return fig

app = App(app_ui, server)

if __name__ == "__main__":
    app.run(port=5001)


[View Application](https://katelassiter2.shinyapps.io/testpy1/)